# Basic experiments

Experiments comparing Random Isolation Similarity Forest to other outlier (anomaly) detection algorithms

In [1]:
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '..')
from data.data_getter import get_numerical_datasets

from sklearn.metrics import roc_auc_score

We will use different outlier detection algorithms to compare to RISF:
* LOF
* ECOD
* Isolation Forest
* HBOS


In [2]:
from pyod.models.ecod import ECOD
from pyod.models.lof import LOF
from pyod.models.iforest import IForest
from pyod.models.hbos import HBOS
# from isf.forest import RandomIsolationSimilarityForest

In [3]:
SEED = 23


We will measure AUC (as a binary classification task of being an outlier) and processing time. We can show plots for every algorithm and the top-N feature importance

In [7]:
datasets = get_numerical_datasets()

In [79]:
from sklearn.metrics import roc_auc_score
from pyod.utils.utility import precision_n_scores
import matplotlib.pyplot as plt

In [26]:
def new_clf(name):
    if name == 'ECOD':
        return ECOD()
    if name == 'LOF':
        return LOF()
    if name == 'IForest':
        return IForest(random_state=SEED)
    if name == 'HBOS':
        return HBOS()
    else:
        raise NotImplementedError()

In [48]:
clfs_names = ['ECOD', 'LOF', 'IForest', 'HBOS']
nan_datasets = []
results = {x: {} for x in clfs_names}

In [53]:
import time
import gc
from typing import Literal, Optional, NoReturn


class Timer:
    """
    timer type can only take the following string values:
    - "performance": the most precise clock in the system.
    - "process": measures the CPU time, meaning sleep time is not measured.
    - "long_running": it is an increasing clock that do not change when the
        date and or time of the machine is changed.
    """

    _counter_start: Optional[int] = None
    _counter_stop: Optional[int] = None

    def __init__(
        self,
        timer_type: Literal["performance", "process", "long_running"] = "performance",
        disable_garbage_collect: bool = True,
    ) -> None:
        self.timer_type = timer_type
        self.disable_garbage_collect = disable_garbage_collect

    def start(self) -> None:
        if self.disable_garbage_collect:
            gc.disable()
        self._counter_start = self._get_counter()

    def stop(self) -> None:
        self._counter_stop = self._get_counter()
        if self.disable_garbage_collect:
            gc.enable()

    @property
    def time_nanosec(self) -> int:
        self._valid_start_stop()
        return self._counter_stop - self._counter_start  # type: ignore

    @property
    def time_sec(self) -> float:
        return self.time_nanosec / 1e9

    def _get_counter(self) -> int:
        counter: int
        if self.timer_type == "performance":
            counter = time.perf_counter_ns()
        elif self.timer_type == "process":
            counter = time.process_time_ns()
        elif self.timer_type == "long_running":
                counter = time.monotonic_ns()
        return counter

    def _valid_start_stop(self) -> Optional[NoReturn]:
        if self._counter_start is None:
            raise ValueError("Timer has not been started.")
        if self._counter_stop is None:
            raise ValueError("Timer has not been stopped.")
        return None

# from https://towardsdatascience.com/execution-times-in-python-ed45ecc1bb4d

In [52]:
timer = Timer(timer_type="long_running")

In [54]:
for name, data in datasets.items():
    print(f'{name}:')
    for clf_name in clfs_names:
        print(clf_name, end=', ')
        clf = new_clf(clf_name)
        timer.start()
        clf.fit(data['X_train'])
        timer.stop()
        
        # get the prediction labels and outlier scores of the training data
        y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
        y_train_scores = clf.decision_scores_  # raw outlier scores

        # get the prediction on the test data
        y_test_pred = clf.predict(data['X_test'])  # outlier labels (0 or 1)
        y_test_scores = clf.decision_function(data['X_test'])  # outlier scores

        
        if np.isnan(y_train_scores).any() or np.isnan(y_train_pred).any():
            nan_datasets.append(name)
            results[clf_name][name] = (np.nan, np.nan, np.nan, np.nan, np.nan) # AUC/ROC, Rank@N for train,test ; fit Time
            continue
        
        roc_train=np.round(roc_auc_score(data['y_train'], y_train_scores), decimals=4)
        prn_train=np.round(precision_n_scores(data['y_train'], y_train_scores), decimals=4)
        roc_test=np.round(roc_auc_score(data['y_test'], y_test_scores), decimals=4)
        prn_test=np.round(precision_n_scores(data['y_test'], y_test_scores), decimals=4)

        results[clf_name][name] = (roc_train, prn_train, roc_test, prn_test, timer.time_sec)
        
    print('\n')

01_ALOI.npz:
ECOD, LOF, IForest, HBOS, 

02_annthyroid.npz:
ECOD, LOF, IForest, HBOS, 

03_backdoor.npz:
ECOD, LOF, IForest, HBOS, 

04_breastw.npz:
ECOD, LOF, IForest, HBOS, 

05_campaign.npz:
ECOD, LOF, IForest, HBOS, 

06_cardio.npz:
ECOD, LOF, IForest, HBOS, 

07_Cardiotocography.npz:
ECOD, LOF, IForest, HBOS, 

08_celeba.npz:
ECOD, LOF, IForest, HBOS, 

09_census.npz:
ECOD, LOF, IForest, HBOS, 

10_cover.npz:
ECOD, LOF, IForest, HBOS, 

11_donors.npz:
ECOD, LOF, IForest, HBOS, 

12_fault.npz:
ECOD, LOF, IForest, HBOS, 

13_fraud.npz:
ECOD, LOF, IForest, HBOS, 

14_glass.npz:
ECOD, LOF, IForest, HBOS, 

15_Hepatitis.npz:
ECOD, LOF, IForest, HBOS, 

16_http.npz:
ECOD, LOF, IForest, HBOS, 

17_InternetAds.npz:
ECOD, LOF, IForest, HBOS, 

18_Ionosphere.npz:
ECOD, LOF, IForest, HBOS, 

19_landsat.npz:
ECOD, LOF, IForest, HBOS, 

20_letter.npz:
ECOD, LOF, IForest, HBOS, 

21_Lymphography.npz:
ECOD, LOF, IForest, HBOS, 

22_magic.gamma.npz:
ECOD, LOF, IForest, HBOS, 

23_mammography.npz:

d:\polibuda\sem5\Isolation-Similarity-Forest\.venv\lib\site-packages\pyod\models\ecod.py:149: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness = np.sign(skew(X, axis=0))


LOF, IForest, HBOS, 

25_musk.npz:
ECOD, LOF, IForest, HBOS, 

26_optdigits.npz:
ECOD, LOF, IForest, HBOS, 

27_PageBlocks.npz:
ECOD, LOF, IForest, HBOS, 

28_pendigits.npz:
ECOD, LOF, IForest, HBOS, 

29_Pima.npz:
ECOD, LOF, IForest, HBOS, 

30_satellite.npz:
ECOD, LOF, IForest, HBOS, 

31_satimage-2.npz:
ECOD, LOF, IForest, HBOS, 

32_shuttle.npz:
ECOD, LOF, IForest, HBOS, 

33_skin.npz:
ECOD, LOF, IForest, HBOS, 

34_smtp.npz:
ECOD, LOF, IForest, HBOS, 

35_SpamBase.npz:
ECOD, 

d:\polibuda\sem5\Isolation-Similarity-Forest\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LOF, IForest, HBOS, 

36_speech.npz:
ECOD, LOF, IForest, HBOS, 

37_Stamps.npz:
ECOD, LOF, IForest, HBOS, 

38_thyroid.npz:
ECOD, LOF, IForest, HBOS, 

39_vertebral.npz:
ECOD, LOF, IForest, HBOS, 

40_vowels.npz:
ECOD, LOF, IForest, HBOS, 

41_Waveform.npz:
ECOD, LOF, IForest, HBOS, 

42_WBC.npz:
ECOD, LOF, IForest, HBOS, 

43_WDBC.npz:
ECOD, LOF, IForest, HBOS, 

44_Wilt.npz:
ECOD, LOF, IForest, HBOS, 

45_wine.npz:
ECOD, LOF, IForest, HBOS, 

46_WPBC.npz:
ECOD, LOF, IForest, HBOS, 

47_yeast.npz:
ECOD, LOF, IForest, HBOS, 



In [97]:
df = pd.DataFrame(results)
df.to_csv('../results/numerical.csv')

In [90]:
times = df.apply(lambda x: x.str[4])
times = times.style.highlight_min(color = 'green', axis = 1)
times

,ECOD,LOF,IForest,HBOS
01_ALOI.npz,0.359000,1.625000,3.031000,0.047000
02_annthyroid.npz,0.015000,0.078000,0.359000,0.000000
03_backdoor.npz,nan,15.547000,32.797000,1.031000
04_breastw.npz,0.016000,0.015000,0.203000,0.000000
05_campaign.npz,0.328000,1.516000,4.953000,0.094000
06_cardio.npz,0.000000,0.047000,0.297000,0.016000
07_Cardiotocography.npz,0.016000,0.032000,0.297000,0.000000
08_celeba.npz,1.204000,344.797000,18.640000,2.547000
09_census.npz,27.328000,377.219000,298.672000,9.062000
10_cover.npz,1.094000,10.375000,12.391000,6.266000


In [91]:
train_roc = df.apply(lambda x: x.str[0])
train_roc = train_roc.style.highlight_max(color = 'green', axis = 1)
train_roc

,ECOD,LOF,IForest,HBOS
01_ALOI.npz,0.525900,0.740100,0.535600,0.533800
02_annthyroid.npz,0.791000,0.719800,0.810600,0.628600
03_backdoor.npz,nan,0.721200,0.758400,0.759000
04_breastw.npz,0.992300,0.446700,0.986900,0.984700
05_campaign.npz,0.769600,0.570400,0.706500,0.789600
06_cardio.npz,0.930200,0.548000,0.921500,0.834800
07_Cardiotocography.npz,0.791700,0.598200,0.697100,0.491200
08_celeba.npz,0.755800,0.424200,0.676900,0.754200
09_census.npz,0.660400,0.544100,0.616100,0.633900
10_cover.npz,0.918400,0.560900,0.851500,0.644800


In [92]:
train_prn = df.apply(lambda x: x.str[1])
train_prn = train_prn.style.highlight_max(color = 'green', axis = 1)
train_prn

,ECOD,LOF,IForest,HBOS
01_ALOI.npz,0.020800,0.143900,0.033100,0.048300
02_annthyroid.npz,0.328900,0.278100,0.323500,0.284200
03_backdoor.npz,nan,0.375100,0.031500,0.022700
04_breastw.npz,0.940100,0.221600,0.910200,0.946100
05_campaign.npz,0.393500,0.086800,0.314700,0.404600
06_cardio.npz,0.504100,0.227600,0.512200,0.447200
07_Cardiotocography.npz,0.512300,0.325200,0.408000,0.269900
08_celeba.npz,0.155800,0.008200,0.103400,0.153000
09_census.npz,0.070200,0.015200,0.048700,0.060500
10_cover.npz,0.160200,0.042600,0.051500,0.045500


In [93]:
test_roc = df.apply(lambda x: x.str[2])
test_roc = test_roc.style.highlight_max(color = 'green', axis = 1)
test_roc

,ECOD,LOF,IForest,HBOS
01_ALOI.npz,0.540700,0.741600,0.552300,0.536300
02_annthyroid.npz,0.778100,0.733500,0.795800,0.613000
03_backdoor.npz,nan,0.696500,0.752300,0.745100
04_breastw.npz,0.989800,0.472200,0.986600,0.982000
05_campaign.npz,0.770100,0.575200,0.703300,0.792400
06_cardio.npz,0.949600,0.602600,0.949500,0.839500
07_Cardiotocography.npz,0.778500,0.603500,0.681300,0.500000
08_celeba.npz,0.760500,0.426500,0.680300,0.758900
09_census.npz,0.657800,0.542100,0.611600,0.632200
10_cover.npz,0.925700,0.567800,0.859300,0.661600


In [94]:
test_prn = df.apply(lambda x: x.str[3])
test_prn = test_prn.style.highlight_max(color = 'green', axis = 1)
test_prn

,ECOD,LOF,IForest,HBOS
01_ALOI.npz,0.039800,0.108400,0.031000,0.042000
02_annthyroid.npz,0.243800,0.287500,0.243800,0.207500
03_backdoor.npz,nan,0.372000,0.028600,0.030000
04_breastw.npz,0.916700,0.208300,0.902800,0.916700
05_campaign.npz,0.391500,0.074700,0.301000,0.392200
06_cardio.npz,0.584900,0.226400,0.603800,0.547200
07_Cardiotocography.npz,0.471400,0.328600,0.385700,0.242900
08_celeba.npz,0.143700,0.006600,0.101200,0.143000
09_census.npz,0.073400,0.014700,0.049200,0.060700
10_cover.npz,0.171100,0.048500,0.054600,0.043700


In [87]:
import dataframe_image as dfi

In [95]:
dfi.export(train_roc, '../results/figures/train_roc.png')
dfi.export(train_prn, '../results/figures/train_prn.png')
dfi.export(test_roc, '../results/figures/test_roc.png')
dfi.export(test_prn, '../results/figures/test_prn.png')
dfi.export(times, '../results/figures/times.png')